In [2]:
import pickle

In [9]:
with open('../data/q2_submodel_results.pkl') as infile:
    results = pickle.load(infile)

In [6]:
with open('../data/q2_submodel_y_preds.pkl') as infile:
    y_preds = pickle.load(infile)

In [7]:
with open('../data/q2_submodel_y_tests.pkl') as infile:
    y_tests = pickle.load(infile)

In [12]:
results.itervalues().next()

{'best_estimator': LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
 'best_params': {},
 'best_score': 0.17094545837951813,
 'result':                              0
 mean_fit_time         0.113809
 mean_score_time     0.00805807
 mean_test_score       0.170945
 mean_train_score      0.202891
 params                      {}
 rank_test_score              1
 split0_test_score     0.170945
 split0_train_score    0.202891
 std_fit_time                 0
 std_score_time               0
 std_test_score               0
 std_train_score              0,
 'rmse': 0.78215492084115734}

In [14]:
r = {}

for categ, result in results.iteritems():
    r[categ] = {
        'CV_test': result['best_score'],
        'rmse': result['rmse'],
        'CV_train': result['result'].loc['mean_train_score'].max()
    }

In [52]:
df = pd.DataFrame(r).T
df.head()

,CV_test,CV_train,rmse
Abandoned Bicycle,5.915372e-02,0.149311,0.595545
Abandoned Vehicles,5.945412e-02,0.082168,0.346471
Animal Generic Request,4.533257e-02,0.171777,0.721914
Bed Bugs,-1.460536e+22,0.081804,0.830620
Building Inspection Request,1.438496e-01,0.158578,0.639544


In [53]:
df.sort_values('rmse').head()

,CV_test,CV_train,rmse
Schedule a Bulk Item Pickup SS,0.083546,0.084689,0.294722
Schedule a Bulk Item Pickup,0.100050,0.102385,0.327968
Notification,0.313788,0.393493,0.329748
OCR Front Desk Interactions,0.059783,0.064095,0.335492
Unsanitary Conditions - Establishment,-0.028027,0.077545,0.344100


In [23]:
df.sort_values('CV_test', ascending=False).head()

,CV_test,CV_train,rmse
Recycling Cart Return,0.445682,0.456628,1.084334
Request for Snow Plowing,0.371975,0.373201,0.563758
Notification,0.313788,0.393493,0.329748
Electrical,0.233042,0.358054,0.619165
Needle Pickup,0.207811,0.324483,0.907098


## Looking at coefs for Recycling Cart Return

Does this have statistically significant coefs, assuming homoskedacity and a linear predictor-response relationship?

## >>>

In [25]:
%load_ext autoreload
%autoreload 2

In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
from tqdm import tqdm

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")
rcParams['figure.figsize'] = 20, 5

import os, sys
sys.path.append(os.path.join(os.path.dirname('.'), "../preprocessing"))
from helper_functions import dummify_cols_and_baselines, make_alphas, remove_outliers_by_type, adjusted_r2, transform_school

In [2]:
df_orig = pd.read_pickle('../data/data_from_remove_from_dataset.pkl')
df_orig.shape

(516406, 40)

In [3]:
df_orig = transform_school(df_orig)
df_orig.shape

../preprocessing/helper_functions.py:58: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  df.school = df.school.str.extract(r'(\d\d?)').astype(int)


(516406, 40)

## Removing outliers

A standard procedure is to remove values further than 3 standard deviations from the mean. Since I have so many low values and some very high values, I anecdotally think that the low values are very likely to be true, and the high values not so much.

So, I will remove values further than 3 SDs from the median, by type.

Ideally, I would take into account the time dimension. I would like to do so given more time.

In [4]:
df_outliers_removed = remove_outliers_by_type(df_orig, y_col='COMPLETION_HOURS_LOG_10')
df_outliers_removed.shape

../preprocessing/helper_functions.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  group[pd.np.abs(group - group.median()) > stds * group.std()] = pd.np.nan
/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:2392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.where(-key, value, inplace=True)


(508653, 40)

I'm removing ~1.5% of my rows.

## Choosing columns

In [5]:
cols_orig_dataset = ['COMPLETION_HOURS_LOG_10', 'TYPE', 'SubmittedPhoto', 'Property_Type', 'Source']
cols_census = [
     'race_black',
     'race_asian',
     'race_hispanic',
     'race_other',
     'poverty_pop_below_poverty_level',
     'earned_income_per_capita',
     'poverty_pop_w_public_assistance',
     'poverty_pop_w_food_stamps',
     'poverty_pop_w_ssi',
     'school',
     'school_std_dev',
     'housing',
     'housing_std_dev',
     'bedroom',
     'bedroom_std_dev',
     'value',
     'value_std_dev',
     'rent',
     'rent_std_dev',
     'income',
     'income_std_dev']
cols_engineered = ['queue_wk', 'queue_wk_open', 'is_description']

In [6]:
df = df_outliers_removed[cols_orig_dataset + cols_census + cols_engineered]
df.shape

(508653, 29)

## Subsetting by Recycling Cart Return

In [7]:
df = df[df.TYPE == 'Recycling Cart Return']
df.shape

(1986, 29)

## Removing NAs for cols like `school_std_dev`

In [26]:
aa = df.isnull().any().reset_index()
nas = aa[aa[0] == True]['index']
print nas

17    housing_std_dev
19    bedroom_std_dev
21      value_std_dev
23       rent_std_dev
25     income_std_dev
Name: index, dtype: object


In [28]:
# this is a bad temporary band-aid
df = df.dropna(subset=nas.tolist())
df.shape

(1975, 29)

## Dummify

In [29]:
cols_to_dummify = [i for i in df.dtypes[df.dtypes == object].index if i != 'TYPE']
cols_to_dummify

['Property_Type', 'Source', 'housing']

In [30]:
df_dummified, baseline_cols = dummify_cols_and_baselines(df, cols_to_dummify)

Intersection is baseline 0 3
Constituent Call is baseline 1 3
rent is baseline 2 3


In [31]:
df_dummified.shape

(1975, 28)

## Running model

In [32]:
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import string
from StringIO import StringIO


In [33]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [34]:
df_dummified.columns = [col.translate(None, string.punctuation).replace(' ', '') if col != 'COMPLETION_HOURS_LOG_10' else col for col in df_dummified.columns]

In [35]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop('COMPLETION_HOURS_LOG_10', axis=1), 
    df_dummified.COMPLETION_HOURS_LOG_10, 
    test_size=0.2, 
    random_state=300
)

In [36]:
col_list = ' + '.join(df_dummified.drop('COMPLETION_HOURS_LOG_10', axis=1))

est = smf.ols(
    'COMPLETION_HOURS_LOG_10 ~ {}'.format(col_list), 
    pd.concat([X_train, y_train], axis=1)).fit()

In [37]:
est.summary().tables[0]

Dep. Variable:,COMPLETION_HOURS_LOG_10,R-squared:,0.439
Model:,OLS,Adj. R-squared:,0.430
Method:,Least Squares,F-statistic:,50.71
Date:,"Thu, 23 Feb 2017",Prob (F-statistic):,1.31e-175
Time:,01:44:24,Log-Likelihood:,-2472.4
No. Observations:,1580,AIC:,4995.
Df Residuals:,1555,BIC:,5129.
Df Model:,24,,
Covariance Type:,nonrobust,,


### Getting adjusted $R^2$ on test set

In [38]:
y_pred = est.predict(X_test)

In [39]:
adjusted_r2(y_test, y_pred, num_features=X_test.shape[1])

0.41378860596518657

In [40]:
mean_squared_error(y_test, y_pred)**0.5

1.1280432616216518

## Interpreting model

Which features are most associated with completion time?

In [44]:
df_results = pd.read_csv(StringIO(est.summary().tables[1].as_csv()), index_col=0).reset_index()
df_results.columns = ['coef_name'] + [i.rstrip().lstrip() for i in df_results.columns][1:]
df_results['coef_abs'] = pd.np.abs(df_results.coef)
df_results = df_results.sort_values('P>|t|')

In [45]:
df_results.shape

(27, 7)

In [47]:
df_results.head(30)

,coef_name,coef,std err,t,P>|t|,[95.0% Conf. Int.],coef_abs
24,queuewkopen,-7.100000e-03,0.000000e+00,-18.621,0.000,-0.008 -0.006,7.100000e-03
23,queuewk,4.000000e-04,1.970000e-05,20.975,0.000,0.000 0.000,4.000000e-04
14,housingstddev,8.109000e-01,2.320000e-01,3.491,0.000,0.355 1.267,8.109000e-01
12,school,8.290000e-02,1.600000e-02,5.141,0.000,0.051 0.115,8.290000e-02
11,povertypopwssi,2.172800e+00,5.220000e-01,4.162,0.000,1.149 3.197,2.172800e+00
4,raceasian,-1.103200e+00,3.560000e-01,-3.099,0.002,-1.801 -0.405,1.103200e+00
26,housingown,-1.625000e-01,7.800000e-02,-2.076,0.038,-0.316 -0.009,1.625000e-01
18,valuestddev,-3.078300e+00,1.587000e+00,-1.940,0.053,-6.191 0.034,3.078300e+00
15,bedroom,6.930000e-02,4.700000e-02,1.472,0.141,-0.023 0.162,6.930000e-02
10,povertypopwfoodstamps,-5.672000e-01,3.950000e-01,-1.435,0.151,-1.342 0.208,5.672000e-01


## Conclusion

Making sub-models improves the R2 and RMSE _for these chosen categories_, which are the ones above the 70th percentile in terms of number of issues, minus a couple ones stated above that messed up the R2.

This R2 and RMSE are 0.59 and 0.69. The ones for the main model are 0.55 and 0.73.

There are prolly more stat sig coefs here as well.

## Next Steps

If I had more time,
- check how the R2 was messed up on those couple of categs. Prolly bc R2 for that model was low, and my preds were way off. what to do about those categs then? either the mean, or the pred from the big model would work.
- group the categs below 70th percentile to give them enough sample size, then run model on them. trial-and-error + domain knowledge as to which categs would work and which wouldn't.